In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd
import rasterio
import rasterio.mask
import matplotlib.pyplot as plt

In [2]:
zone_information = pd.read_csv("/gpfsnyu/home/yw9871/New data/Original_data/zone-information.csv")
#test = zone_information.drop_duplicates(subset=['district'])
zone_information.head(2)

,TAZID,longitude,latitude,charge_count,area,perimeter,name,district
0,1066,113.910301,22.521287,40,1577892.982,5119.3013,南山,南山区
1,1068,113.918680,22.517837,271,1405268.834,5460.2638,南山,南山区


In [9]:
gdp = pd.read_excel("/gpfsnyu/home/yw9871/New data/Original_data/GDP.xlsx")
gdp.head(2)

,district,GDP2022,GDP2023
0,福田区,5514.49,5704.55
1,罗湖区,2630.19,2808.19


In [12]:
gdp['weight2022'] = gdp['GDP2022'] / gdp['GDP2022'].sum()
gdp['weight2023'] = gdp['GDP2023'] / gdp['GDP2023'].sum()
gdp['gdp_weight'] = (gdp['weight2022'] + gdp['weight2023']) / 2
gdp.head(2)

,district,GDP2022,GDP2023,weight2022,weight2023,gdp_weight
0,福田区,5514.49,5704.55,0.170709,0.165437,0.168073
1,罗湖区,2630.19,2808.19,0.081421,0.081440,0.081431


In [15]:
ev_volume = pd.read_excel("/gpfsnyu/home/yw9871/New data/Original_data/EV volume.xlsx")
ev_volume.columns = ['time','city','district','ev_volume']
ev_volume['volume_weight'] = ev_volume['ev_volume'] / ev_volume['ev_volume'].sum()
ev_volume.head(2)

,time,city,district,ev_volume,volume_weight
0,截至2024年12月底,深圳,福田区,240772,0.169951
1,截至2024年12月底,深圳,罗湖区,184777,0.130426


In [19]:
district = pd.merge(gdp, ev_volume, on='district', how='left')
district = district[['district', 'gdp_weight', 'volume_weight']]
district.head(2)

,district,gdp_weight,volume_weight
0,福田区,0.168073,0.169951
1,罗湖区,0.081431,0.130426


In [21]:
zone = pd.merge(zone_information, district, on='district', how='left')
zone = zone[['TAZID', 'gdp_weight', 'volume_weight']]
zone.head(2)

,TAZID,gdp_weight,volume_weight
0,1066,0.248592,0.127608
1,1068,0.248592,0.127608


In [22]:
zone.to_csv("/gpfsnyu/home/yw9871/New data/Results/gdp_volume.csv",index=None)